
The scripts and notes below were developed/written based on exercises or "totally copied" from [Teclado Code](https://github.com/tecladocode/complete-python-course/tree/master/course_contents/13_async_development)


# Asynchronous Python Development

- __Synchronous:__ actions that happen one after another. Programming as we've seen it until now is synchronous, because each line executes after the previous one.
- __Asynchronous:__ actions that don't necessary happen after one another, or that can happen in arbitrary order ("without synchrony").
- __Concurrency:__ The ability of our programs to run things in different order every time the program runs, without affecting the final outcome.
- __Parallelism:__ Running two or more things at the same time.
- __Thread:__ A line of code execution that can run in one of your computer's cores.


## Dining Philosopher

- __Case:__ 5 philosophers and 2 forks who are hungry. The are able to eat just using 2 forks.  
- __Solution:__ If there is a waiter (master, orquestrator), he can get the 2 forks and send them to the philosophers in a limited time for them to eat.
It will be possible to feed 2 philopher by time, and the others would need to wait for their turn (__time slicing__).
Even decreasing the time, it will never be possible for all of them to eat at the same time.  
- __Limited resources__ -> Forks

## Processes and threads

- __Processor__: Each processor has a number of cores.

- __Cores:__ 
Generally, each unit of core has 4 cores inside it.
Each core can work independently and communicate with each other, performing mathematical operations.
Cores are "philosophers" with resources (forks).

- __Threads:__
They are line of code execution.
Each one can run in one core by a time.
Threads are "philosophers waiting to eat".

- __Processes:__
They manage everything (resources, which can be cores + network, hard drive, etc) that is necessary to run one or more threads (which runs things).
Time slicing.
The OS is responsible to save the current status (checkpoint) of the thread to manage it.

- __GIL (Global Interpreter Lock) - Asynchronous Python:__ 
Lauching a Python app, it will get a new Python process.
Python doesn't run 2 threads in one process at same time.
Each Python process creates a key resource(GIL) and each thread acquires that resource.

- __Multiple Pythons__
It is possible to run multiple processes, which means that each one will creates its own GIL, and execute one thread.
However it is expensive to communicate between 2 processes.

__What's the point of multiple threads in Python?__  

Reduce waiting time!

Ex: If you need to request a parameter for a user and execute some mathematical processing. The first thing is going to request the waiting time. With GIL and multiple threads the time will be reduced, because it will consume the CPU just for what the computer needs to execute.



In [4]:
import time
from threading import Thread

####### USING SINGLE THREAD

def ask_user():
    start = time.time()
    user_input = input('Enter your name: ')
    greet = f'Hello, {user_input}'
    print(greet)
    print('ask_user: ', time.time() - start)

def complex_calculation():
    print('Started calculating...')
    start = time.time()
    [x**2 for x in range(20000)]
    print('complex_calculation: ', time.time() - start)


# With a single thread, we can do one at a time—e.g.
start = time.time()
ask_user()
complex_calculation()
print('Single thread total time: ', time.time() - start, '\n\n')


####### USING TWO THREADS

# With two threads, we can do them both at once.
thread = Thread(target=complex_calculation)
thread2 = Thread(target=ask_user)

start = time.time()

# Start both threads.
thread.start()
thread2.start()

# Make the main thread (the whole code) wait for the 2 threads to print the final total time
# They are blocking operations because their behaviour.
thread.join()
thread2.join()

print('Two thread total time: ', time.time() - start)


Enter your name: Jose
Hello, Jose
ask_user:  1.4345934391021729
Started calculating...
complex_calculation:  0.01759815216064453
Single thread total time:  1.4548828601837158 


Started calculating...
complex_calculation:  0.019173622131347656
Enter your name: Jose
Hello, Jose
ask_user:  1.0603420734405518
Two thread total time:  1.0833075046539307


A most elegant way to write the Thread code is using the concurrent.futures method, because it uses a __Context Manager__ as the following example.

__Important note:__ It is possible to write commands inside the code to kill the threads between the start and the waiting process (blocking), but it SHOULDN'T be done, because it may create a deadlock, killing the GIL, which will make the code "wait forever" for the next step.

In [4]:
import time
from concurrent.futures import ThreadPoolExecutor

####### USING SINGLE THREAD

def ask_user():
    start = time.time()
    user_input = input('Enter your name: ')
    greet = f'Hello, {user_input}'
    print(greet)
    print('ask_user: ', time.time() - start)

def complex_calculation():
    print('Started calculating...')
    start = time.time()
    [x**2 for x in range(20000)]
    print('complex_calculation: ', time.time() - start)


# With a single thread, we can do one at a time—e.g.
start = time.time()
ask_user()
complex_calculation()
print('Single thread total time: ', time.time() - start, '\n\n')


####### USING TWO THREADS
# With two threads, we can do them both at once
start = time.time()

# Create a pool of threads (in this case, 2), then submit to start, forcing the main thread to wait the 2 new ones
with ThreadPoolExecutor(max_workers=2) as pool:
    pool.submit(complex_calculation)
    pool.submit(ask_user)

# The pool.shutdown() is implicit into the Context Manager, that is the reason why we don't need to call it

print('Two thread total time: ', time.time() - start)

Enter your name: Ana
Hello, Ana
ask_user:  4.666213035583496
Started calculating...
complex_calculation:  0.01547098159790039
Single thread total time:  4.683517932891846 


Started calculating...
complex_calculation:  0.013892173767089844
Enter your name: Ana
Hello, Ana
ask_user:  1.5530469417572021
Two thread total time:  1.5554237365722656


### <a href=https://www.linkedin.com/in/jmilhomem/>br.linkedin.com/in/jmilhomem</a> ###